In [1]:
import sys
import numpy as np
import pandas as pd
from sklearn.metrics import r2_score

### Custom files ###
dir_gp_constr = 'C:\\Users\\tln229\\Downloads\\Python Tests\\Constrained GP_2\\GP_2\\gp_constr\\'
sys.path.append(dir_gp_constr)

# Import
from GPConstr.model import GPmodel, Constraint
from GPConstr.kern import kernel_Matern52

Unable to determine R home: [WinError 2] The system cannot find the file specified
R[write to console]: Loading required package: mvtnorm



Loading constrained GP module from C:\Users\tln229\Downloads\Python\1. Building\Finals\2. Supply Air model
Loading R wrapper...


R[write to console]: Loading required package: Matrix

R[write to console]: Loading required package: stats4

R[write to console]: Loading required package: gmm

R[write to console]: Loading required package: sandwich

R[write to console]: 
Attaching package: 'TruncatedNormal'


R[write to console]: The following objects are masked from 'package:tmvtnorm':

    dtmvnorm, dtmvt, ptmvnorm, ptmvt, rtmvnorm, rtmvt


R[write to console]: The following objects are masked from 'package:mvtnorm':

    pmvnorm, pmvt




Running R from rpy2: R version 4.2.2 (2022-10-31 ucrt)


In [2]:
def get_diagnostics_df(x_test, y_test, mean, var):
    """ Put test data in a dataframe"""
    df = pd.DataFrame(x_test)
    df.columns = ['x' + str(i+1) for i in range(x_test.shape[1])]
    df['y_true'] = y_test
    df['y_mean'] = mean
    df['y_var'] = var
    return df

In [3]:
def constant_function(val):
    def fun(x):
        return np.array([val]*x.shape[0])
    return fun

In [4]:
# ---------- DATA ----------
def create_data(n_train, n_Xv):
    df = pd.read_csv('C:/Users/tln229/Downloads/Python/1. Building/data/HVAC_B90_102_exp_10m_20210424.csv')

    # TRAIN SET
    Tca_k1   = np.array(df['ahu_supply_temp']).reshape(-1,1)[1: n_train+1]
    Tsa_k    = np.array(df['supply_discharge_temp']).reshape(-1,1)[0: n_train]
    Tsa_k1   = np.array(df['supply_discharge_temp']).reshape(-1,1)[1: n_train+1]
    valve_k2 = np.array(df['htg_valve_position']).reshape(-1,1)[2: n_train+2]

    x_train = np.concatenate((Tca_k1, Tsa_k, Tsa_k1, valve_k2), axis=1)
    y_train = np.array(df['supply_discharge_temp'])[2: n_train+2]

    # Xv
    x1v = np.linspace(53, 57, n_Xv).reshape(-1, 1)
    x2v = np.linspace(56, 96, n_Xv).reshape(-1, 1)
    x3v = np.linspace(56, 96, n_Xv).reshape(-1, 1)
    x4v = np.linspace(0, 100, n_Xv).reshape(-1, 1)
    Xv  = np.concatenate((x1v, x2v, x3v, x4v), axis = 1)

    # TEST SET
    Tca_k1_t   = np.array(df['ahu_supply_temp']).reshape(-1,1)[n_train+1: 1600+1]
    Tsa_k_t    = np.array(df['supply_discharge_temp']).reshape(-1,1)[n_train: 1600]
    Tsa_k1_t   = np.array(df['supply_discharge_temp']).reshape(-1,1)[n_train+1: 1600+1]
    valve_k2_t = np.array(df['htg_valve_position']).reshape(-1,1)[n_train+2: 1600+2]

    x_test = np.concatenate((Tca_k1_t, Tsa_k_t, Tsa_k1_t, valve_k2_t), axis=1)
    y_test = np.array(df['supply_discharge_temp'])[n_train+2: 1600+2]

    return x_train, y_train, Xv, x_test, y_test

In [5]:
df_result = pd.DataFrame({'n_train':[], 'n_Xv':[], 'R2':[]})

for n_train in ([32, 64, 128]):
    for n_Xv in ([5, 10, 20]):
        # DATA
        x_train, y_train, Xv, x_test, y_test = create_data(n_train=n_train, n_Xv=n_Xv)

        # MODEL
        ker = kernel_Matern52(variance = 1, lengthscale = [1]*x_train.shape[1])
        model = GPmodel(kernel = ker, likelihood = 1, mean = 0)

        model.X_training = x_train
        model.Y_training = y_train

        model.optimize(include_constraint = False, fix_likelihood = False)

        # CONSTRAINTS
        model.constr_deriv = [
            Constraint(LB = constant_function(0), UB = constant_function(float('Inf'))),
            Constraint(LB = constant_function(float('-Inf')), UB = constant_function(float('Inf'))),
            Constraint(LB = constant_function(float('-Inf')), UB = constant_function(float('Inf'))),
            Constraint(LB = constant_function(0), UB = constant_function(float('Inf')))
        ]

        model.constr_deriv[0].Xv = Xv
        model.constr_deriv[1].Xv = Xv
        model.constr_deriv[2].Xv = Xv
        model.constr_deriv[3].Xv = Xv

        model.constr_likelihood = 1E-6

        # PREDICT
        mean, _, _, _, _, _ = model.calc_posterior_constrained(x_test, num_samples = 5000, save_samples = 500)
        mean = np.array(mean).flatten()
        df_result.loc[len(df_result)] = [n_train, n_Xv, r2_score(y_test, mean)]

..Running optimization for unconstrained GP ... DONE - Total time: 0.071 seconds
..Running calculation of K_w ... SKIP - (cached)
..Running calculation of Cholesky factor for K_w ... SKIP - (cached)
..Running preparation step 1 - dependence on (XS, X) ... DONE - time: 0.070 seconds
..Running preparation step 2 - dependence on (XV, X) ... DONE - time: 0.001 seconds
..Running preparation step 3 - dependence on (XS, XV, X) ... DONE - time: 0.016 seconds
..sampling 5000 times from truncated constraint distribution C~|C, Y DONE - time: 0.060 seconds
..sampling 5000 times from constrained GP f*|C, Y DONE - time: 3.681 seconds
..computing statistics from samples DONE - time: 0.203 seconds
 DONE - Total time: 4.032 seconds
..Running optimization for unconstrained GP ... DONE - Total time: 0.076 seconds
..Running calculation of K_w ... SKIP - (cached)
..Running calculation of Cholesky factor for K_w ... SKIP - (cached)
..Running preparation step 1 - dependence on (XS, X) ... DONE - time: 0.075 

In [6]:
print(df_result)

   n_train  n_Xv        R2
0     32.0   5.0  0.754234
1     32.0  10.0  0.754656
2     32.0  20.0  0.752235
3     64.0   5.0  0.988724
4     64.0  10.0  0.988669
5     64.0  20.0  0.988561
6    128.0   5.0  0.989679
7    128.0  10.0  0.989655
8    128.0  20.0  0.989645
